In [2]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [3]:
import pyspark
from pyspark.sql import SparkSession


spark = SparkSession.builder.master("yarn")\
.appName("ch26Classification").getOrCreate()


#.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
# .config("spark.driver.memory", "1g")\
# .config("spark.executor.memory", "3g")\
# .config("spark.executor.cores", "4")\
# .config("spark.executor.instances", "4")\


#!/usr/bin/env bash

# This file is sourced when running various Spark programs.
# Copy it as spark-env.sh and edit that to configure Spark for your site.

# Options read in YARN client mode
#SPARK_EXECUTOR_INSTANCES="2" #Number of workers to start (Default: 2)
#SPARK_EXECUTOR_CORES="1" #Number of cores for the workers (Default: 1).
#SPARK_EXECUTOR_MEMORY="1G" #Memory per Worker (e.g. 1000M, 2G) (Default: 1G)
#SPARK_DRIVER_MEMORY="512M" #Memory for Master (e.g. 1000M, 2G) (Default: 512 Mb)
#SPARK_YARN_APP_NAME="spark" #The name of your application (Default: Spark)
#SPARK_YARN_QUEUE="default" #The hadoop queue to use for allocation requests (Default: default)
#SPARK_YARN_DIST_FILES="" #Comma separated list of files to be distributed with the job.
#SPARK_YARN_DIST_ARCHIVES="" #Comma separated list of archives to be distributed with the job.

In [4]:
sc = spark.sparkContext

In [5]:
for x in sc._conf.getAll():
    if "/proxy/" in x[1]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1533622723243_18505


In [6]:
data_dir = "/user/kranthidr/dataSets/spark-guide/"
path = data_dir + "binary-classification"

In [7]:
bInput = spark.read.format("parquet").load(path)\
  .selectExpr("features", "cast(label as double) as label")

In [8]:
bInput.show(10,False)

+--------------+-----+
|features      |label|
+--------------+-----+
|[3.0,10.1,3.0]|1.0  |
|[1.0,0.1,-1.0]|0.0  |
|[1.0,0.1,-1.0]|0.0  |
|[2.0,1.1,1.0] |1.0  |
|[2.0,1.1,1.0] |1.0  |
+--------------+-----+



In [9]:
# COMMAND ----------
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()

# _init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#         maxIter=100, regParam=0.0, elasticNetParam=0.0, tol=1e-6, fitIntercept=True,                  
#         threshold=0.5, thresholds=None, probabilityCol="probability",                  
#         rawPredictionCol="rawPrediction", standardization=True, weightCol=None,                  
#         aggregationDepth=2, family="auto",                  
#         lowerBoundsOnCoefficients=None, upperBoundsOnCoefficients=None,                  
#         lowerBoundsOnIntercepts=None, upperBoundsOnIntercepts=None):

In [11]:
print(lr.explainParams()) # see all parameters

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal wi

In [12]:
lrModel = lr.fit(bInput)

In [13]:
# COMMAND ----------
print(lrModel.coefficients)
print(lrModel.intercept)

[6.848741326854929,0.3535658901019745,14.814900276915923]
-10.22569586448109


In [14]:
# COMMAND ----------
summary = lrModel.summary

In [15]:
print(summary.areaUnderROC)

1.0


In [16]:
summary.roc.show()

+---+------------------+
|FPR|               TPR|
+---+------------------+
|0.0|               0.0|
|0.0|0.3333333333333333|
|0.0|               1.0|
|1.0|               1.0|
|1.0|               1.0|
+---+------------------+



In [17]:
summary.pr.show()

+------------------+---------+
|            recall|precision|
+------------------+---------+
|               0.0|      1.0|
|0.3333333333333333|      1.0|
|               1.0|      1.0|
|               1.0|      0.6|
+------------------+---------+



In [18]:
# COMMAND ----------

summary.objectiveHistory

[0.6730116670092563,
 0.5042829330409727,
 0.36356862066874396,
 0.1252407018038337,
 0.08532556611276214,
 0.035504876415730455,
 0.018196494508571255,
 0.008817369922959136,
 0.004413673785392143,
 0.002194038351234709,
 0.0010965641148080857,
 0.000547657551985314,
 0.00027376237951490126,
 0.0001368465223657475,
 6.841809037070595e-05,
 3.420707791038497e-05,
 1.7103176664232043e-05,
 8.551470106426904e-06,
 4.275703677941461e-06,
 2.1378240117781303e-06,
 1.068856405465203e-06,
 5.34260020257524e-07,
 2.668135105897439e-07,
 1.3204627865316843e-07,
 6.768401481686428e-08,
 3.314547718487037e-08,
 1.6151438837494788e-08,
 8.309350118269286e-09]

In [19]:
# COMMAND ----------

from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier()

# Init docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#                          probabilityCol="probability", rawPredictionCol="rawPrediction",                  
#                          maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0,                  
#                          maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10, impurity="gini",                  
#                          seed=None)

In [20]:
print(dt.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name. (default: label)
maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree.

In [21]:
dtModel = dt.fit(bInput)

In [22]:
# COMMAND ----------
from pyspark.ml.classification import RandomForestClassifier
rfClassifier = RandomForestClassifier()


# Init docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#                          probabilityCol="probability", rawPredictionCol="rawPrediction",                  
#                          maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0,                  
#                          maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10, impurity="gini",                  
#                          numTrees=20, featureSubsetStrategy="auto", seed=None, subsamplingRate=1.0)

In [24]:
print(rfClassifier.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: auto)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name. (default: label)
maxBins: Max

In [25]:
trainedModel = rfClassifier.fit(bInput)

In [26]:
# COMMAND ----------
from pyspark.ml.classification import GBTClassifier
gbtClassifier = GBTClassifier()

# init docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#                         maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0,                  
#                         maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10,                  
#                         lossType="logistic", maxIter=20, stepSize=0.1, seed=None, subsamplingRate=1.0)


In [27]:
print(gbtClassifier.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
labelCol: label column name. (default: label)
lossType: Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic (default: logistic)
maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (>= 0)

In [28]:
trainedModel = gbtClassifier.fit(bInput)

In [29]:
# COMMAND ----------
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

# Init docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  p
#                          robabilityCol="probability", rawPredictionCol="rawPrediction", smoothing=1.0,                 
#                          modelType="multinomial", thresholds=None, weightCol=None)

In [31]:
print(nb.explainParams())

featuresCol: features column name. (default: features)
labelCol: label column name. (default: label)
modelType: The model type which is a string (case-sensitive). Supported options: multinomial (default) and bernoulli. (default: multinomial)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
smoothing: The smoothing parameter, should be >= 0, default is 1.0 (default: 1.0)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is 

In [32]:
trainedModel = nb.fit(bInput.where("label != 0"))

In [33]:
# COMMAND ----------
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [34]:
model = trainedModel

In [35]:
out = model.transform(bInput)\
  .select("prediction", "label")\
  .rdd.map(lambda x: (float(x[0]), float(x[1])))

In [36]:
metrics = BinaryClassificationMetrics(out)

In [40]:
# COMMAND ----------
# print(metrics.areaUnderPR)
# print(metrics.areaUnderROC)
# print("Receiver Operating Characteristic")

#Eror in pyspark-lab kernel
# Py4JJavaError: An error occurred while calling o438.areaUnderPR.
# : org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 236.0 failed 4 times, 
#             most recent failure: Lost task 0.3 in stage 236.0 (TID 415, wn05.itversity.com, executor 3): 
#                     java.io.IOException: 
#                         Cannot run program "/home/kranthidr/.virtualenvs/pyspark-lab/bin/python3.5": error=2,
#                             No such file or directory

In [42]:
# metrics.roc.toDF().show()
# #AttributeError: 'BinaryClassificationMetrics' object has no attribute 'roc'

In [41]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [43]:
bceval = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label', metricName='areaUnderROC')

# Init signature: BinaryClassificationEvaluator(
#rawPredictionCol='rawPrediction', labelCol='label', metricName='areaUnderROC')
# Docstring:     
# .. note:: Experimental

In [44]:
transed = trainedModel.transform(bInput)

In [45]:
transed.columns

['features', 'label', 'rawPrediction', 'probability', 'prediction']

In [54]:
ml_bmetrics = bceval.evaluate(transed)

# # y4JJavaError: An error occurred while calling o772.evaluate.
# # : org.apache.spark.SparkException: Job aborted due to stage failure:
# #             Task 0 in stage 483.0 failed 4 times, most recent failure: 
# #                 Lost task 0.3 in stage 483.0 (TID 864, wn03.itversity.com, executor 2): 
# #                     java.lang.ArrayIndexOutOfBoundsException: 1
# # 	at org.apache.spark.ml.linalg.DenseVector.apply(Vectors.scala:462)

Py4JJavaError: An error occurred while calling o492.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 248.0 failed 4 times, most recent failure: Lost task 0.3 in stage 248.0 (TID 451, wn05.itversity.com, executor 3): java.lang.ArrayIndexOutOfBoundsException: 1
	at org.apache.spark.ml.linalg.DenseVector.apply(Vectors.scala:462)
	at org.apache.spark.ml.evaluation.BinaryClassificationEvaluator$$anonfun$1.apply(BinaryClassificationEvaluator.scala:82)
	at org.apache.spark.ml.evaluation.BinaryClassificationEvaluator$$anonfun$1.apply(BinaryClassificationEvaluator.scala:81)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:193)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:63)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:96)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapT

In [47]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [48]:
mceval = MulticlassClassificationEvaluator()

# Init signature: MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
# Docstring:   

In [49]:
transed = trainedModel.transform(bInput)

In [50]:
transed.columns

['features', 'label', 'rawPrediction', 'probability', 'prediction']

In [51]:
ml_bmetrics = mceval.evaluate(transed)

In [52]:
ml_bmetrics.real

0.2285714285714286